# Continuous Control

---




# DDPG Agent 

# Import necessary modules

In [1]:
# --- General NN and math libraries --- #
import numpy as np
import torch

# --- For Measuring performance --- #
from collections import deque
from statistics import mean, stdev
import matplotlib.pyplot as plt

# --- For Loading the environment --- #
from ENV import environment_loader

In [2]:
# --- For Loading the agent --- #
# We don't need to reimport the agent everytime we change one of it's values
%load_ext autoreload 
%autoreload 2
from AGENT import DDPG_Agent

ModuleNotFoundError: No module named 'autoreload '

# Load Environment
Environment is loaded with a helper function in ENV.py
The helper function returns the envrionment object, the state size and action size, the brain's name and how many agents there are in the environment

In [ ]:

env, state_size, action_size, brain_name, num_agents = environment_loader("Reacher_Single/Reacher.exe", no_graphics = False)


# Train Agent

In [ ]:
def DDPG_train(agent, env, max_t=100, num_episodes = 1, print_every=100, existing_score=[]):
    score_history = existing_score
    print("Initial Score History length: ", len(score_history))
    scores_deque = deque(maxlen=print_every)
    last_running_mean = float('-inf')

    for episode in range(num_episodes):
        env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
        agent.reset()                                          # reset the noise in the agent
        states = env_info.vector_observations                  # get the current state (for each agent)
        scores = np.zeros(num_agents)                          # initialize the score (for each agent)
        score = 0

        for i in range(max_t):
            actions = agent.act(states[0])                     # select an action (for each agent)
            env_info = env.step(actions)[brain_name]           # send all actions to the environment
            
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished

            agent.step(states[0], actions, rewards[0], next_states[0], dones[0])

            score += env_info.rewards[0]

            states = next_states                               # roll over states to next time step
            if np.any(dones):                                  # exit loop if episode finished
                break
        

        #returns_in_episode = np.mean(scores)
        returns_in_episode = score
        scores_deque.append(returns_in_episode)
        score_history.append(returns_in_episode)
        if episode > print_every:
            if mean(scores_deque) > last_running_mean:
                    print("")
                    print('Last {} was better, going to save it'.format(print_every))
                    torch.save(new_agent.actor_local.state_dict(), 'checkpoint_actor.pth')
                    torch.save(new_agent.critic_local.state_dict(), 'checkpoint_critic.pth')
                    last_running_mean = mean(scores_deque)

        print("\r", 'Total score (averaged over agents) {} episode: {} | \tAvarage in last {} is {}'.format(episode, returns_in_episode, print_every, np.mean(scores_deque)), end="")
        ##print("\r", 'Scores in {} episode: {}'.format(episode, scores), end="")
    

    return score_history

In [ ]:
import os.path

filename="checkpoint_actor.pth"
path = ""

def fileAtLocation(filename, path):
    return os.path.exists(path + filename)

def load_previous(new_agent):
    loaded_agent = new_agent
    if fileAtLocation(filename, path):
        print("Found previous trained Agent, going to load them!")
        loaded_agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
        loaded_agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))
    return loaded_agent

In [ ]:
new_agent = DDPG_Agent(state_size, action_size, 1, actor_hidden = [256, 256], critic_hidden=[256, 256])


In [ ]:

score_history = DDPG_train(new_agent, env, max_t = 1001, num_episodes = 150)
#print(score_history)

# Display and Save results

In [ ]:
print(score_history)
print(len(score_history))
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(score_history)+1), score_history)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()
#plt.savefig("")

In [ ]:
parsed = [float(i) for i in new_score_history]
with open("last_score.csv", 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(score_history)

# Continue last saved run
The next to cells 
- Import the scores of the last run
- If we saved weights of the Neural Net load those
- Continue training from last place

In [ ]:
import csv

with open('last_score.csv', newline='') as f:
    reader = csv.reader(f)
    read_score_history = list(reader)[0]

In [ ]:
new_agent = load_previous(new_agent)

In [ ]:
new_score_history = DDPG_train(new_agent, env, max_t = 1001, num_episodes = 150, existing_score=read_score_history)

In [ ]:
# print(new_score_history)
parsed = [float(i) for i in new_score_history]
# print(parsed)
# print(len(new_score_history))
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(parsed)+1), parsed)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
with open("last_score.csv", 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(parsed)

# See how the agent performs

In [ ]:
%load_ext autoreload
%autoreload 2
from AGENT import DDPG_Agent

new_agent = DDPG_Agent(state_size, action_size, 5)
new_agent = load_previous(new_agent)

In [ ]:
def DDPG_eval(agent, env, max_t=100):
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    episode_length = 0                                     # measure number of steps allowed before done flag
    for i in range(max_t):
        # actions = np.random.randn(num_agents, action_size) # select an action (for each agent); randomly!
        actions =  np.random.randn(num_agents, action_size) if agent is None else agent.act(states[0]) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to the environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        episode_length +=1
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Episode length is {}'.format(episode_length))
    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

In [ ]:
DDPG_eval(new_agent, env, max_t = 100000000000)

In [ ]:
env.close()